In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import gym
from torch.distributions.categorical import Categorical


In [ ]:
class ActorCritic(nn.Module):
    def __init__(self, obs_dim, act_dim, hidden_size=64):
        """
        obs_dim  : размерность вектора наблюдения
        act_dim  : размерность (число возможных действий)
        hidden_size: размер скрытых слоёв
        """
        super(ActorCritic, self).__init__()
        
        # Актер (Policy)
        self.actor = nn.Sequential(
            nn.Linear(obs_dim, hidden_size),
            nn.ReLU(),
            nn.Linear(hidden_size, hidden_size),
            nn.ReLU(),
            nn.Linear(hidden_size, act_dim)  # logits на act_dim действий
        )
        
        # Критик (Value function)
        self.critic = nn.Sequential(
            nn.Linear(obs_dim, hidden_size),
            nn.ReLU(),
            nn.Linear(hidden_size, hidden_size),
            nn.ReLU(),
            nn.Linear(hidden_size, 1)        # ценность состояния (скаляр)
        )
    
    def forward(self, obs):
        """
        Он просто есть, потому что мы обязаны привести реализацию этого класса 
        """
        raise NotImplementedError
    
    def get_action_and_value(self, obs):
        """
        Для заданного obs возвращаем:
          - action (выбор из act_dim)
          - log_prob(action)
          - value (V(obs))
          - распределение dist
        """
        logits = self.actor(obs)
        dist = Categorical(logits=logits)
        action = dist.sample()
        value = self.critic(obs).squeeze(-1)  # shape: [batch_size]
        return action, dist.log_prob(action), dist, value
